In [ ]:
import trimesh
import random
from collections import defaultdict
import copy
import matplotlib.pyplot as plt
import numpy as np
from geometry.helper import voxelize, voxel_extents, voxel_neighbor, voxel_dir, voxel_face

In [ ]:
def check(code):
    mxz = max([x[2] for x in code])
    if mxz < 4: return False

    cnt = 0
    for x, y, z in code:
        if z == 0: cnt += 1
    if cnt < 3: return False
    return True

# dirs = [(1, 0, 0), (-1, 0, 0), (0, 1, 0), (0, -1, 0), (0, 0, 1), (0, 0, -1)]
dirs = [(1, 0, 0), (0, 1, 0), (0, 0, 1), (-1, 0, 0), (0, -1, 0)]
def rec(pos, rest):
    global cur, types
    if rest == 0:
        return 1
    tot = 0
    for dx, dy, dz in dirs:
        x, y, z = pos[0] + dx, pos[1] + dy, pos[2] + dz
        if (x, y, z) in cur:
            continue
        # structure constraints
        # if z > 0 and (x, y, z-1) not in cur:
        #     continue
        cur.append((x, y, z))
        tot += rec((x, y, z), rest - 1)
        if check(cur):
            types.append(copy.deepcopy(cur))
        cur.pop()
    return tot

In [ ]:
cur = []
types = []

rec((0, 0, 0), 10)

In [ ]:
len(types)

In [ ]:
count_box = defaultdict(lambda: 0)

for t in types:
    count_box[len(t)] += 1

plt.pie(count_box.values(), labels=count_box.keys());

In [ ]:
def mesh_wireframe(mesh: trimesh.Trimesh, angle_threshold=-0.0, color=(0, 0, 0)):
    is_large_angle = mesh.face_adjacency_angles >= angle_threshold
    large_angle_edges = trimesh.load_path(mesh.vertices[mesh.face_adjacency_edges[is_large_angle]])

    mesh_outline = mesh.outline()

    all_edges = trimesh.path.util.concatenate([large_angle_edges, mesh_outline])

    for edge in all_edges.entities:
        edge.color = color

    return all_edges

In [ ]:
def get_box_type(code):
    boxes = []
    wireframes = []
    for x, y, z in code:
        box = trimesh.creation.box(extents=[1, 1, 1])
        box.apply_translation((x, y, z))
        wireframes.append(mesh_wireframe(box, 0.1))
        color = trimesh.visual.random_color()
        for fid in range(len(box.faces)):
            box.visual.face_colors[fid] = color
        boxes.append(box)
    return boxes, wireframes

In [ ]:
rnd = random.randint(0, len(types)-1)
m, bt = voxelize(types[rnd])
b, w = get_box_type(types[rnd])

trimesh.Scene().show()

In [ ]:
scene = trimesh.Scene()
m, bt = voxelize(types[rnd])
b, w = get_box_type(types[rnd])
scene.add_geometry(voxel_neighbor(m, bt))
scene.export('box.obj');


In [ ]:
rnd = random.randint(0, len(types)-1)
b, w = get_box_type(types[rnd])
mn, mx = voxel_extents(types[rnd])
outer_box = trimesh.creation.box(extents=[mx[i] - mn[i] + 1 for i in range(3)])
outer_box.apply_translation([(mn[i] + mx[i]) / 2 for i in range(3)])
outer_wireframe = mesh_wireframe(outer_box, 0.1)
trimesh.Scene(b+w+[outer_wireframe]).show()

In [ ]:
def distance(p1, p2):
    return sum([abs(p1[i] - p2[i]) for i in range(3)])

def count_inner(code):
    tot = 0
    for i, c1 in enumerate(code):
        for j, c2 in enumerate(code):
            if distance(c1, c2) == 1:
                print(c1, c2, distance(c1, c2))
                tot += 1

    return tot

6*len(types[rnd]) - count_inner(types[rnd])

In [ ]:
def occupy(code):
    mn, mx = voxel_extents(code)
    volume = (mx[0] - mn[0] + 1) * (mx[1] - mn[1] + 1) * (mx[2] - mn[2] + 1)
    return len(code) / volume

profiles = {}
for i, code in enumerate(types):
    mn, mx = voxel_extents(code)
    profiles[i] = {
        'occupy': occupy(code),
        'min_dimension': min([mx[i] - mn[i] + 1 for i in range(3)]),
    }

In [ ]:
filtered_types = []
for i in profiles:
    if profiles[i]['occupy'] >= 0.5 and profiles[i]['min_dimension'] > 1:
        filtered_types.append(types[i])

print(len(filtered_types))


rnd = random.randint(0, len(filtered_types)-1)
b, w = get_box_type(filtered_types[rnd])
mn, mx = voxel_extents(filtered_types[rnd])
outer_box = trimesh.creation.box(extents=[mx[i] - mn[i] + 1 for i in range(3)])
outer_box.apply_translation([(mn[i] + mx[i]) / 2 for i in range(3)])
outer_wireframe = mesh_wireframe(outer_box, 0.1)
trimesh.Scene(b+w+[outer_wireframe]).show()


In [ ]:
box = trimesh.creation.box(extents=(1, 1, 1))


print(len(box.facets))
for i in range(12):
    box.visual.face_colors[i] = trimesh.visual.random_color()



In [ ]:
def discrete_box(code):
    d = 0.05

    boxes = []
    for i in range(3):
        for j in range(2):
            for dx, dy in [(0, 0), (0, 1), (1, 1), (1, 0)]:
                box = trimesh.creation.box(extents = voxel_dir(d)[i])
                hole = trimesh.creation.box(extents = voxel_dir(d, 0.25)[i])
                box = box.difference(hole)
                box.apply_translation(voxel_dir(d)[i]/2 + voxel_dir(1-d, 0)[i]*j + voxel_dir(0, x=dx, y=dy)[i])
                boxes.append(box)
    wireframes = []
    for b in boxes:
        b.apply_translation(code)
        wireframes.append(mesh_wireframe(b, 0.1))
        # color = trimesh.visual.random_color()
        # for i in b.faces:
        #     b.visual.face_colors[i] = color
    return boxes, wireframes



type_ = [(0, -1, 0), (0, -2, 0), (-1, -2, 0), (-1, -2, 1), (0, -2, 1), (0, -2, 2)]
scene = trimesh.Scene()
for c in type_:
    boxes, wireframes = discrete_box(c)
    scene.add_geometry(boxes)
    scene.add_geometry(wireframes)

scene.show()



In [ ]:
types[rnd]